### Applied Data Science Capstone Assignment 2 :Segmenting and Clustering Neighborhoods in Toronto - Part 1

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.
Build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

To create the dataframe:

-The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

-Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

-More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

-If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [1]:
# Import requiered libraries

import requests
import lxml.html as lh
import pandas as pd

In [2]:
# Save data url in variable
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Create a handle page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under variable doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
#Validate the length of the first couple rows (they should have 3 columns)
[len(T) for T in tr_elements[:2]]

[3, 3]

In [4]:
# Parse 1st row as header
tr_elements = doc.xpath('//tr')

#Create an empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


In [6]:
# Append each header to a tuple, along with an empty list

# Data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is different from 3, the data is not from table 
    if len(T)!=3:
        break
    
    #Assing "i" to the index of column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

# Create pandas dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

# Visualize 1st 5 rows
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [15]:
#Rearrange and rename the columns (BEWARE: postcode should have e.g. "M1A" as values)
# based on https://stackoverflow.com/questions/13148429/how-to-change-the-order-of-dataframe-columns

df.columns = ['Borough', 'Neighbourhood','Postcode']
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [16]:
#Remove "\n" from table values
df = df.replace('\n',' ', regex=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [20]:
# Drop cells with Borough Not assigned
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

# Reset index, drop previous index
df = df.reset_index(drop=True)

#Check how table looks
df.head(20)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Not assigned
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [23]:
#Consolidate neighbourhoods based on equal Postcode and Borough
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']

#Check how table looks
df.head(20)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge ,Malvern ,Rouge ,Malvern"
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union ,Highla..."
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill ,Guildwood ,..."
3,M1G,Scarborough,"Woburn ,Woburn"
4,M1H,Scarborough,"Cedarbrae ,Cedarbrae"
5,M1J,Scarborough,"Scarborough Village ,Scarborough Village"
6,M1K,Scarborough,"East Birchmount Park ,Ionview ,Kennedy Park ,E..."
7,M1L,Scarborough,"Clairlea ,Golden Mile ,Oakridge ,Clairlea ,Gol..."
8,M1M,Scarborough,"Cliffcrest ,Cliffside ,Scarborough Village Wes..."
9,M1N,Scarborough,"Birch Cliff ,Cliffside West ,Birch Cliff ,Clif..."


In [25]:
# Remove any space at the beggining of the string
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

#Assign Borough values to Neignbourhood where vlaue is "Not assigned"
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

#Check how table looks
df.head(20)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge ,Malvern ,Rouge ,Malvern"
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union ,Highla..."
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill ,Guildwood ,..."
3,M1G,Scarborough,"Woburn ,Woburn"
4,M1H,Scarborough,"Cedarbrae ,Cedarbrae"
5,M1J,Scarborough,"Scarborough Village ,Scarborough Village"
6,M1K,Scarborough,"East Birchmount Park ,Ionview ,Kennedy Park ,E..."
7,M1L,Scarborough,"Clairlea ,Golden Mile ,Oakridge ,Clairlea ,Gol..."
8,M1M,Scarborough,"Cliffcrest ,Cliffside ,Scarborough Village Wes..."
9,M1N,Scarborough,"Birch Cliff ,Cliffside West ,Birch Cliff ,Clif..."


In [26]:
# Finally, review the shape of the data frame
df.shape

(103, 3)

In [30]:
# Use %store magic in order to save the recently created dataframe (in order to use it on the part 2 notebook)
# https://stackoverflow.com/questions/13148429/how-to-change-the-order-of-dataframe-columns

%store df

Stored 'df' (DataFrame)
